# Receptive Fields in the Visual Cortex

## Overview

Welcome to the second section of our Visual Neuroscience module! In this notebook, we'll explore receptive fields in the primary visual cortex (V1), focusing on their structure, properties, and how they process visual information. 

In the previous section, we examined the retina and lateral geniculate nucleus (LGN), where we encountered center-surround receptive fields. Now, we'll see how these early visual signals are transformed into more complex features in the cortex through specialized receptive fields that are selective for orientation and spatial frequency.

### What we'll cover:
- Structure and properties of V1 receptive fields
- Gabor filters as models of V1 receptive fields
- Spatial frequency and orientation tuning
- Receptive field mapping techniques
- Changes in receptive field properties across the visual hierarchy

## Setting Up

Let's import the libraries we'll need for this notebook.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation, cm
from mpl_toolkits.mplot3d import Axes3D
import scipy.signal as signal
import sys
import ipywidgets as widgets
from IPython.display import HTML, display

# Add the utils package to the path
sys.path.append('../../..')
try:
    from motionenergy.utils import stimuli_generation, visualization
except ImportError:
    print("Note: utils modules not found. This is expected if you haven't implemented them yet.")

# For interactive plots
%matplotlib inline

# Set plotting style
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 12

## 1. From Retina/LGN to V1: The Emergence of Orientation Selectivity

In the previous section, we explored center-surround receptive fields in the retina and LGN. These cells respond best to spots of light and are not selective for orientation. However, when we move to the primary visual cortex (V1), we find neurons with fundamentally different response properties.

### The Classic Hubel and Wiesel Experiments

In the late 1950s and early 1960s, David Hubel and Torsten Wiesel conducted groundbreaking experiments recording from individual neurons in the visual cortex of cats. They discovered that V1 neurons respond most strongly to bars or edges with specific orientations in specific locations in the visual field.

This discovery led to a fundamental insight about visual processing: the visual system builds increasingly complex representations through hierarchical processing, with each stage extracting more sophisticated features from the visual input.

### The Feed-Forward Model of Orientation Selectivity

How do orientation-selective receptive fields emerge from center-surround inputs? Hubel and Wiesel proposed a simple feed-forward model:

1. Multiple center-surround LGN cells with receptive fields arranged along a line provide input to a V1 cell
2. When a bar of light with the correct orientation falls on these aligned LGN receptive fields, they all fire simultaneously
3. The V1 cell integrates these inputs and fires, exhibiting orientation selectivity

Let's visualize this model:

In [ ]:
def create_lgn_receptive_field(size=64, sigma_center=2, sigma_surround=4, position=(0, 0), polarity='ON'):
    """Create a center-surround receptive field like those in LGN."""
    x = np.linspace(-size/2, size/2, size)
    y = np.linspace(-size/2, size/2, size)
    X, Y = np.meshgrid(x, y)
    
    # Shift position
    X = X - position[0]
    Y = Y - position[1]
    
    # Create center and surround components
    center = np.exp(-(X**2 + Y**2) / (2 * sigma_center**2))
    surround = np.exp(-(X**2 + Y**2) / (2 * sigma_surround**2))
    
    # Normalize
    center = center / np.max(center)
    surround = surround / np.max(surround)
    
    # Combine center and surround based on polarity
    if polarity == 'ON':
        rf = center - 0.5 * surround
    else:  # OFF
        rf = -center + 0.5 * surround
    
    return rf

def visualize_v1_from_lgn_model():
    # Create a row of LGN cells (for simplicity, all ON-center)
    positions = [(i*8 - 16, 0) for i in range(5)]
    lgn_rfs = [create_lgn_receptive_field(position=pos) for pos in positions]
    
    # Create a simple stimulus: a bar at 45 degrees
    size = 64
    stimulus = np.zeros((size, size))
    for i in range(size):
        if 22 <= i <= 42:
            stimulus[i, i] = 1
    
    # Calculate LGN responses
    lgn_responses = [np.sum(rf * stimulus) for rf in lgn_rfs]
    
    # Visualize
    fig, axs = plt.subplots(2, 3, figsize=(15, 10))
    
    # Show the stimulus
    axs[0, 0].imshow(stimulus, cmap='gray')
    axs[0, 0].set_title('Stimulus (45° Bar)')
    axs[0, 0].axis('off')
    
    # Show LGN receptive fields and responses
    cmap = plt.cm.RdBu_r
    combined_rf = np.zeros_like(lgn_rfs[0])
    
    for i, (rf, response) in enumerate(zip(lgn_rfs, lgn_responses)):
        # Add to combined RF (for visualization only, with scaling)
        combined_rf += rf * 0.5  
        
        # Show individual RFs
        if i < 2:  # Just show two examples to save space
            axs[0, i+1].imshow(rf, cmap=cmap)
            axs[0, i+1].set_title(f'LGN Cell {i+1} RF')
            axs[0, i+1].axis('off')
    
    # Show combined LGN input
    im = axs[1, 0].imshow(combined_rf, cmap=cmap)
    axs[1, 0].set_title('Combined LGN Input')
    axs[1, 0].axis('off')
    plt.colorbar(im, ax=axs[1, 0])
    
    # Show LGN responses
    bars = axs[1, 1].bar(range(len(lgn_responses)), lgn_responses)
    axs[1, 1].set_title('LGN Cell Responses')
    axs[1, 1].set_xlabel('LGN Cell')
    axs[1, 1].set_ylabel('Response')
    axs[1, 1].set_xticks(range(len(lgn_responses)))
    axs[1, 1].set_ylim(-0.1, 0.5)
    
    # Show V1 cell response
    v1_response = sum(lgn_responses)
    axs[1, 2].bar(0, v1_response, color='green')
    axs[1, 2].set_title('V1 Cell Response')
    axs[1, 2].set_xticks([0])
    axs[1, 2].set_xticklabels(['V1 Cell'])
    axs[1, 2].set_ylabel('Response')
    axs[1, 2].set_ylim(0, 2)
    
    plt.tight_layout()
    plt.show()
    
    return lgn_responses, v1_response

# Visualize the feed-forward model
lgn_responses, v1_response = visualize_v1_from_lgn_model()

This simple feed-forward model provides an intuitive framework for understanding how orientation selectivity could emerge. However, in reality, V1 receptive fields are more complex and involve additional mechanisms like lateral inhibition and feedback connections.

Modern research has shown that V1 receptive fields arise from a combination of:
- Feed-forward connections from the LGN
- Lateral connections between V1 neurons
- Feedback connections from higher visual areas
- Complex inhibitory and excitatory interactions

These mechanisms contribute to the rich and diverse set of receptive field properties found in V1 neurons.

## 2. Gabor Filters as Models of V1 Receptive Fields

One of the most successful models for V1 receptive fields is the Gabor filter, which provides a good approximation of the spatial structure of orientation-selective cells in the visual cortex.

A Gabor filter is the product of a sinusoidal plane wave and a Gaussian envelope. Mathematically, it can be expressed as:

$$g(x, y; \lambda, \theta, \psi, \sigma, \gamma) = \exp\left(-\frac{x'^2 + \gamma^2 y'^2}{2\sigma^2}\right) \cos\left(2\pi\frac{x'}{\lambda} + \psi\right)$$

where:
- $x' = x\cos\theta + y\sin\theta$
- $y' = -x\sin\theta + y\cos\theta$
- $\lambda$ = wavelength (spatial frequency)
- $\theta$ = orientation
- $\psi$ = phase offset
- $\sigma$ = standard deviation of the Gaussian envelope
- $\gamma$ = spatial aspect ratio

Let's implement a Gabor filter function and visualize how different parameters affect its appearance:

In [ ]:
def gabor_filter(size=64, wavelength=10, orientation=0, phase=0, sigma=10, aspect_ratio=0.5):
    """Create a Gabor filter.
    
    Parameters:
    -----------
    size : int
        Size of the filter (pixels)
    wavelength : float
        Wavelength of the sinusoidal component (pixels)
    orientation : float
        Orientation (radians)
    phase : float
        Phase offset (radians)
    sigma : float
        Standard deviation of the Gaussian envelope
    aspect_ratio : float
        Spatial aspect ratio of the Gaussian (y/x)
        
    Returns:
    --------
    gabor : ndarray
        2D array with the Gabor filter
    """
    # Create coordinates
    x = np.linspace(-size/2, size/2, size)
    y = np.linspace(-size/2, size/2, size)
    X, Y = np.meshgrid(x, y)
    
    # Rotation
    X_rot = X * np.cos(orientation) + Y * np.sin(orientation)
    Y_rot = -X * np.sin(orientation) + Y * np.cos(orientation)
    
    # Gaussian envelope
    gaussian = np.exp(-(X_rot**2 + (aspect_ratio * Y_rot)**2) / (2 * sigma**2))
    
    # Sinusoidal carrier
    sinusoid = np.cos(2 * np.pi * X_rot / wavelength + phase)
    
    # Gabor filter
    gabor = gaussian * sinusoid
    
    return gabor

# Create and visualize a basic Gabor filter
basic_gabor = gabor_filter()

plt.figure(figsize=(10, 8))
plt.imshow(basic_gabor, cmap='RdBu_r')
plt.colorbar(label='Filter Value')
plt.title('Gabor Filter (Default Parameters)')
plt.axis('off')
plt.show()

Now, let's create an interactive visualization to explore how changing parameters affects the Gabor filter:

In [ ]:
@widgets.interact(
    wavelength=widgets.FloatSlider(min=4, max=20, step=1, value=10, description='Wavelength:'),
    orientation=widgets.FloatSlider(min=0, max=np.pi, step=np.pi/12, value=0, description='Orientation:'),
    phase=widgets.FloatSlider(min=0, max=2*np.pi, step=np.pi/6, value=0, description='Phase:'),
    sigma=widgets.FloatSlider(min=4, max=20, step=1, value=10, description='Sigma:'),
    aspect_ratio=widgets.FloatSlider(min=0.2, max=1.0, step=0.1, value=0.5, description='Aspect Ratio:')
)
def update_gabor(wavelength, orientation, phase, sigma, aspect_ratio):
    """Update the Gabor filter visualization based on parameter changes."""
    # Create the Gabor filter
    gb = gabor_filter(
        wavelength=wavelength,
        orientation=orientation,
        phase=phase,
        sigma=sigma,
        aspect_ratio=aspect_ratio
    )
    
    # Plot the filter
    fig, axs = plt.subplots(1, 2, figsize=(15, 6))
    
    # 2D image
    im = axs[0].imshow(gb, cmap='RdBu_r')
    axs[0].set_title('Gabor Filter')
    axs[0].axis('off')
    plt.colorbar(im, ax=axs[0])
    
    # 3D surface plot
    x = np.linspace(-32, 32, 64)
    y = np.linspace(-32, 32, 64)
    X, Y = np.meshgrid(x, y)
    ax = axs[1]
    ax = plt.subplot(1, 2, 2, projection='3d')
    surf = ax.plot_surface(X, Y, gb, cmap='coolwarm', linewidth=0, antialiased=True)
    ax.set_title('3D Visualization')
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Filter Value')
    
    plt.tight_layout()
    plt.show()

### Why Gabor Filters?

Gabor filters provide an excellent model for V1 receptive fields for several reasons:

1. **Biological accuracy**: Receptive fields of V1 simple cells closely resemble Gabor functions when mapped experimentally

2. **Optimal joint resolution**: Gabor filters achieve the theoretical lower bound for joint uncertainty in the spatial and frequency domains (similar to the uncertainty principle in quantum mechanics)

3. **Feature extraction**: They effectively extract oriented edges and textures, which are fundamental features in natural images

4. **Mathematical tractability**: Gabor functions have well-understood mathematical properties that make them useful for modeling and analysis

These properties make Gabor filters not only good models for biological vision but also useful tools in computer vision and image processing.

## 3. Orientation and Spatial Frequency Tuning

Two key properties of V1 neurons are their selectivity for orientation and spatial frequency. Let's explore these properties in detail.

### Orientation Tuning

Orientation tuning describes how a neuron's response varies with the orientation of a stimulus. Most V1 neurons respond preferentially to stimuli with a specific orientation and less to stimuli with other orientations. The relationship between stimulus orientation and neural response can be plotted as an orientation tuning curve.

Let's simulate the orientation tuning of a V1 neuron using our Gabor filter model:

In [ ]:
def grating_stimulus(size=64, orientation=0, spatial_freq=0.1, phase=0):
    """Create a sine wave grating stimulus.
    
    Parameters:
    -----------
    size : int
        Size of the stimulus (pixels)
    orientation : float
        Orientation of the grating (radians)
    spatial_freq : float
        Spatial frequency of the grating (cycles/pixel)
    phase : float
        Phase of the grating (radians)
        
    Returns:
    --------
    grating : ndarray
        2D array with the grating stimulus
    """
    # Create coordinates
    x = np.linspace(-size/2, size/2, size)
    y = np.linspace(-size/2, size/2, size)
    X, Y = np.meshgrid(x, y)
    
    # Rotate coordinates
    X_rot = X * np.cos(orientation) + Y * np.sin(orientation)
    
    # Create grating
    grating = np.sin(2 * np.pi * X_rot * spatial_freq + phase)
    
    return grating

def compute_response(stimulus, receptive_field):
    """Compute the response of a neuron with the given receptive field to a stimulus.
    
    Parameters:
    -----------
    stimulus : ndarray
        2D array with the stimulus
    receptive_field : ndarray
        2D array with the receptive field
        
    Returns:
    --------
    response : float
        Response of the neuron
    """
    # Normalize receptive field
    rf_normalized = receptive_field / np.sqrt(np.sum(receptive_field**2))
    
    # Compute response (dot product)
    response = np.sum(stimulus * rf_normalized)
    
    # Apply a simple rectification (to simulate firing rates can't be negative)
    response = max(0, response)
    
    return response

def plot_orientation_tuning():
    """Plot orientation tuning curve for a V1 neuron."""
    # Create a Gabor filter as our model V1 receptive field
    preferred_orientation = np.pi / 4  # 45 degrees
    rf = gabor_filter(orientation=preferred_orientation, wavelength=8, sigma=8)
    
    # Test orientations
    orientations = np.linspace(0, np.pi, 24)  # 0 to 180 degrees
    responses = []
    
    # Compute responses to gratings at different orientations
    for theta in orientations:
        stimulus = grating_stimulus(orientation=theta, spatial_freq=1/8)  # Match wavelength
        response = compute_response(stimulus, rf)
        responses.append(response)
    
    # Normalize responses
    responses = np.array(responses)
    responses = responses / np.max(responses)
    
    # Plot tuning curve
    fig, axs = plt.subplots(1, 3, figsize=(18, 6))
    
    # Plot receptive field
    axs[0].imshow(rf, cmap='RdBu_r')
    axs[0].set_title('V1 Receptive Field\n(Preferred: 45°)')
    axs[0].axis('off')
    
    # Plot example stimulus
    example_stimulus = grating_stimulus(orientation=preferred_orientation, spatial_freq=1/8)
    axs[1].imshow(example_stimulus, cmap='gray')
    axs[1].set_title('Example Stimulus\n(Orientation: 45°)')
    axs[1].axis('off')
    
    # Plot tuning curve
    orientations_deg = orientations * 180 / np.pi
    axs[2].plot(orientations_deg, responses, 'o-', linewidth=2)
    axs[2].axvline(preferred_orientation * 180 / np.pi, color='r', linestyle='--', alpha=0.5, label='Preferred')
    axs[2].set_xlabel('Orientation (degrees)')
    axs[2].set_ylabel('Normalized Response')
    axs[2].set_title('Orientation Tuning Curve')
    axs[2].set_xlim(0, 180)
    axs[2].set_xticks(np.arange(0, 181, 30))
    axs[2].set_ylim(0, 1.05)
    axs[2].grid(True, alpha=0.3)
    axs[2].legend()
    
    plt.tight_layout()
    plt.show()
    
    return orientations_deg, responses

# Plot orientation tuning curve
orientations, responses = plot_orientation_tuning()

### Orientation Tuning Width and Selectivity

The width of the orientation tuning curve indicates how selective a neuron is for orientation. Neurons with narrow tuning curves respond only to a small range of orientations, while those with broader tuning curves are less selective.

V1 neurons vary in their orientation selectivity, with some showing very sharp tuning (high selectivity) and others showing broader tuning (lower selectivity). This diversity of tuning properties allows the visual system to represent orientation information with different levels of precision.

### Spatial Frequency Tuning

In addition to orientation selectivity, V1 neurons are also tuned for spatial frequency, which determines the scale at which they detect features. Spatial frequency is measured in cycles per degree of visual angle and refers to how rapidly a visual pattern repeats over space.

Let's simulate the spatial frequency tuning of a V1 neuron:

In [ ]:
def plot_spatial_frequency_tuning():
    """Plot spatial frequency tuning curve for a V1 neuron."""
    # Create a Gabor filter as our model V1 receptive field
    preferred_wavelength = 8  # pixels
    preferred_sf = 1 / preferred_wavelength  # cycles/pixel
    rf = gabor_filter(wavelength=preferred_wavelength, sigma=8)
    
    # Test spatial frequencies (as wavelengths)
    wavelengths = np.logspace(np.log10(2), np.log10(32), 20)  # 2 to 32 pixels
    spatial_freqs = 1 / wavelengths  # cycles/pixel
    responses = []
    
    # Compute responses to gratings at different spatial frequencies
    for sf in spatial_freqs:
        stimulus = grating_stimulus(orientation=0, spatial_freq=sf)
        response = compute_response(stimulus, rf)
        responses.append(response)
    
    # Normalize responses
    responses = np.array(responses)
    responses = responses / np.max(responses)
    
    # Plot tuning curve
    fig, axs = plt.subplots(1, 3, figsize=(18, 6))
    
    # Plot receptive field
    axs[0].imshow(rf, cmap='RdBu_r')
    axs[0].set_title(f'V1 Receptive Field\n(Preferred SF: {preferred_sf:.3f} cycles/pixel)')
    axs[0].axis('off')
    
    # Plot example stimuli
    # Create a subplot with three different spatial frequencies
    sf_examples = [spatial_freqs[0], preferred_sf, spatial_freqs[-1]]
    sf_stimuli = [grating_stimulus(orientation=0, spatial_freq=sf) for sf in sf_examples]
    
    # Combine into one image with dividing lines
    example_img = np.zeros((64, 64 * 3))
    for i, stim in enumerate(sf_stimuli):
        example_img[:, i*64:(i+1)*64] = stim
    
    axs[1].imshow(example_img, cmap='gray')
    axs[1].set_title('Example Stimuli\n(Low, Preferred, High Spatial Frequency)')
    axs[1].axis('off')
    
    # Plot tuning curve
    axs[2].semilogx(spatial_freqs, responses, 'o-', linewidth=2)
    axs[2].axvline(preferred_sf, color='r', linestyle='--', alpha=0.5, label='Preferred')
    axs[2].set_xlabel('Spatial Frequency (cycles/pixel)')
    axs[2].set_ylabel('Normalized Response')
    axs[2].set_title('Spatial Frequency Tuning Curve')
    axs[2].set_xlim(spatial_freqs.min(), spatial_freqs.max())
    axs[2].set_ylim(0, 1.05)
    axs[2].grid(True, alpha=0.3)
    axs[2].legend()
    
    plt.tight_layout()
    plt.show()
    
    return spatial_freqs, responses

# Plot spatial frequency tuning curve
spatial_freqs, sf_responses = plot_spatial_frequency_tuning()

### Bandpass Filtering Properties

The spatial frequency tuning curve shows that V1 neurons act as bandpass filters, responding most strongly to a specific range of spatial frequencies and less to frequencies that are either too low or too high. This bandpass filtering property allows the visual system to analyze images at multiple spatial scales.

Different V1 neurons are tuned to different spatial frequencies, allowing the visual system to simultaneously represent fine details (high spatial frequencies) and coarse structure (low spatial frequencies) in the visual scene.

## 4. Receptive Field Mapping Techniques

How do neuroscientists determine the structure and properties of receptive fields in the visual cortex? Several techniques have been developed to map receptive fields experimentally. Let's explore some of these methods.

### Hubel and Wiesel's Method: Bars and Edges

The classic approach used by Hubel and Wiesel involved presenting bars or edges at different positions and orientations while recording from a single neuron. By systematically varying the stimulus parameters and noting when the neuron fired, they could map out its receptive field structure.

This labor-intensive method provided the first insights into the orientation selectivity of V1 neurons.

In [ ]:
def simulate_classic_rf_mapping():
    """Simulate the classic Hubel and Wiesel receptive field mapping."""
    # Create a Gabor filter as our "ground truth" receptive field
    true_rf = gabor_filter(size=32, wavelength=8, orientation=np.pi/4, sigma=6)  # 45 degrees
    
    # Define the positions and orientations to test
    positions = np.linspace(-12, 12, 5)
    orientations = np.linspace(0, np.pi, 6)  # 0 to 180 degrees
    
    # Initialize response matrix
    responses = np.zeros((len(positions), len(orientations)))
    
    # Test each position and orientation
    for i, pos in enumerate(positions):
        for j, theta in enumerate(orientations):
            # Create a bar stimulus
            stimulus = np.zeros((32, 32))
            for k in range(32):
                for l in range(32):
                    # Rotate coordinates
                    x_rot = (l - 16) * np.cos(theta) + (k - 16) * np.sin(theta)
                    if abs(x_rot - pos) < 1.5:  # Bar width = 3 pixels
                        stimulus[k, l] = 1
            
            # Compute response
            response = compute_response(stimulus, true_rf)
            responses[i, j] = response
    
    # Normalize responses
    responses = responses / np.max(responses)
    
    # Visualize
    fig, axs = plt.subplots(1, 3, figsize=(18, 6))
    
    # Show the true receptive field
    axs[0].imshow(true_rf, cmap='RdBu_r')
    axs[0].set_title('True Receptive Field')
    axs[0].axis('off')
    
    # Show the measured responses
    im = axs[1].imshow(responses, cmap='viridis', extent=[
        0, 180,  # x-axis: orientation in degrees
        positions[-1], positions[0]  # y-axis: position
    ])
    axs[1].set_xlabel('Orientation (degrees)')
    axs[1].set_ylabel('Position')
    axs[1].set_title('Measured Responses')
    plt.colorbar(im, ax=axs[1], label='Normalized Response')
    
    # Show an example stimulus
    best_i, best_j = np.unravel_index(np.argmax(responses), responses.shape)
    best_pos = positions[best_i]
    best_ori = orientations[best_j]
    
    # Create the best stimulus
    best_stimulus = np.zeros((32, 32))
    for k in range(32):
        for l in range(32):
            x_rot = (l - 16) * np.cos(best_ori) + (k - 16) * np.sin(best_ori)
            if abs(x_rot - best_pos) < 1.5:
                best_stimulus[k, l] = 1
    
    axs[2].imshow(best_stimulus, cmap='gray')
    axs[2].set_title(f'Optimal Stimulus\n(Orientation: {best_ori * 180 / np.pi:.1f}°, Position: {best_pos:.1f})')
    axs[2].axis('off')
    
    plt.tight_layout()
    plt.show()
    
    return responses

# Simulate classic receptive field mapping
classic_responses = simulate_classic_rf_mapping()

### Reverse Correlation: White Noise Analysis

A more modern approach is reverse correlation, also known as white noise analysis or spike-triggered averaging. This method involves:

1. Presenting a random stimulus (white noise) to the visual system
2. Recording the neuron's spikes in response to this stimulus
3. Averaging the stimuli that preceded each spike

This average gives an estimate of the receptive field structure without requiring prior assumptions about its properties.

Let's simulate this technique:

In [ ]:
def simulate_reverse_correlation():
    """Simulate reverse correlation (white noise) receptive field mapping."""
    # Create a Gabor filter as our "ground truth" receptive field
    true_rf = gabor_filter(size=32, wavelength=6, orientation=np.pi/4, sigma=5)
    
    # Create sequence of white noise stimuli
    n_frames = 10000
    noise_stimuli = np.random.normal(0, 1, (n_frames, 32, 32))
    
    # Compute responses
    responses = np.array([compute_response(stim, true_rf) for stim in noise_stimuli])
    
    # Select stimuli that elicited strong responses (simulating spikes)
    threshold = np.percentile(responses, 95)  # Top 5% of responses
    spike_indices = np.where(responses > threshold)[0]
    spike_triggered_stimuli = noise_stimuli[spike_indices]
    
    # Compute spike-triggered average (STA)
    sta = np.mean(spike_triggered_stimuli, axis=0)
    
    # Visualize
    fig, axs = plt.subplots(1, 3, figsize=(15, 5))
    
    # Show the true receptive field
    axs[0].imshow(true_rf, cmap='RdBu_r')
    axs[0].set_title('True Receptive Field')
    axs[0].axis('off')
    
    # Show example noise stimulus
    axs[1].imshow(noise_stimuli[0], cmap='gray')
    axs[1].set_title('Example Noise Stimulus')
    axs[1].axis('off')
    
    # Show the estimated receptive field (STA)
    axs[2].imshow(sta, cmap='RdBu_r')
    axs[2].set_title('Estimated RF (Spike-Triggered Average)')
    axs[2].axis('off')
    
    plt.tight_layout()
    plt.show()
    
    # Calculate the correlation between true and estimated RF
    correlation = np.corrcoef(true_rf.flatten(), sta.flatten())[0, 1]
    print(f"Correlation between true and estimated RF: {correlation:.3f}")
    
    return sta

# Simulate reverse correlation mapping
estimated_rf = simulate_reverse_correlation()

### Advanced Techniques

Modern neuroscience employs several more sophisticated techniques for mapping receptive fields:

1. **Subspace Reverse Correlation**: Uses structured stimuli (like Gabor patches) instead of white noise

2. **Receptive Field Estimation using GLMs**: Uses generalized linear models to estimate receptive fields from responses to complex stimuli

3. **Calcium Imaging**: Allows simultaneous recording from many neurons, providing maps of receptive fields across a neural population

4. **fMRI Retinotopic Mapping**: Maps receptive fields at a coarser scale in humans using non-invasive imaging

These techniques have revealed the rich diversity of receptive field properties in the visual cortex and how these properties change across the visual hierarchy.

## 5. Receptive Field Changes Across the Visual Hierarchy

As visual information flows from the retina through the visual cortex and into higher visual areas, receptive fields undergo systematic changes in size and complexity. Let's explore these changes and their functional significance.

### Receptive Field Size Progression

One of the most consistent changes across the visual hierarchy is an increase in receptive field size:

In [ ]:
def visualize_rf_size_progression():
    """Visualize how receptive field size increases across the visual hierarchy."""
    # Create receptive fields of different sizes (simulating different visual areas)
    size = 128
    
    # Create retina/LGN center-surround RF
    retina_rf = create_lgn_receptive_field(size=size, sigma_center=3, sigma_surround=6, polarity='ON')
    
    # Create V1 simple cell RF (small Gabor)
    v1_rf = gabor_filter(size=size, wavelength=8, orientation=np.pi/4, sigma=6, aspect_ratio=0.6)
    
    # Create V2 RF (larger Gabor, potentially more complex)
    v2_rf = gabor_filter(size=size, wavelength=12, orientation=np.pi/4, sigma=12, aspect_ratio=0.7)
    
    # Create V4 RF (even larger)
    v4_rf = gabor_filter(size=size, wavelength=18, orientation=np.pi/4, sigma=20, aspect_ratio=0.8)
    
    # Create IT RF (large and less structured, simulated as a large Gaussian)
    x = np.linspace(-size/2, size/2, size)
    y = np.linspace(-size/2, size/2, size)
    X, Y = np.meshgrid(x, y)
    it_rf = np.exp(-(X**2 + Y**2) / (2 * 30**2))
    
    # Visualize
    fig, axs = plt.subplots(1, 5, figsize=(15, 5))
    regions = ['Retina/LGN', 'V1', 'V2', 'V4', 'IT']
    rfs = [retina_rf, v1_rf, v2_rf, v4_rf, it_rf]
    cmaps = ['gray', 'RdBu_r', 'RdBu_r', 'RdBu_r', 'gray']
    
    for i, (region, rf, cmap) in enumerate(zip(regions, rfs, cmaps)):
        axs[i].imshow(rf, cmap=cmap)
        axs[i].set_title(f'{region}\nRF Size: {np.sqrt(np.sum(rf**2)):.1f}')
        axs[i].axis('off')
        
        # Add a circle to visualize RF size
        if i > 0:  # Skip for Retina/LGN since it's not a simple size measure
            sigma = [6, 12, 20, 30][i-1]
            circle = plt.Circle((size/2, size/2), 2*sigma, fill=False, edgecolor='red')
            axs[i].add_patch(circle)
    
    plt.tight_layout()
    plt.show()
    
# Visualize RF size progression
visualize_rf_size_progression()

### Functional Significance of Changing Receptive Field Properties

These changes in receptive field properties across the visual hierarchy serve important functional roles:

1. **Integration of Information**: Larger receptive fields in higher areas integrate information from larger regions of visual space

2. **Feature Complexity**: More complex properties in higher areas allow detection of more complex visual features

3. **Invariance**: Larger and more complex receptive fields contribute to invariant recognition (recognizing objects regardless of exact position, size, etc.)

4. **Spatial and Feature Hierarchies**: The visual system simultaneously builds both a spatial hierarchy (from small to large) and a feature hierarchy (from simple to complex)

These hierarchical changes allow the visual system to transform the pixel-like representation in the retina into increasingly abstract and useful representations of the visual world.

## 6. Applying Gabor Filters to Natural Images

To better understand how V1 receptive fields process visual information, let's apply Gabor filters to natural images. This will demonstrate how these filters extract oriented features from complex scenes.

In [ ]:
def create_natural_image(size=128):
    """Create a simple natural-like image with edges and textures."""
    # Start with a blank image
    image = np.zeros((size, size))
    
    # Add some geometric shapes
    # Rectangle
    image[20:60, 30:70] = 0.8
    
    # Triangle
    for i in range(30):
        width = int(i * 30 / 30)
        image[80+i, 80-width:80+width] = 0.6
    
    # Add some texture
    texture = np.random.normal(0, 0.1, (size, size))
    image += texture
    
    # Add a gradient background
    x = np.linspace(0, 1, size)
    X, Y = np.meshgrid(x, x)
    gradient = 0.2 * (X + Y) / 2
    image += gradient
    
    # Ensure values are in [0, 1]
    image = np.clip(image, 0, 1)
    
    return image

def apply_gabor_filter_bank(image, n_orientations=6, n_wavelengths=3):
    """Apply a bank of Gabor filters to an image and visualize the responses."""
    # Create a bank of Gabor filters with different orientations and wavelengths
    orientations = np.linspace(0, np.pi, n_orientations, endpoint=False)
    wavelengths = [4, 8, 16][:n_wavelengths]  # Different scales
    
    # Apply each filter and store the responses
    responses = []
    filters = []
    
    for wavelength in wavelengths:
        for orientation in orientations:
            # Create the filter
            gabor = gabor_filter(
                size=32,  # Smaller filter for efficiency
                wavelength=wavelength,
                orientation=orientation,
                sigma=wavelength * 0.75,  # Scale sigma with wavelength
                aspect_ratio=0.6
            )
            filters.append(gabor)
            
            # Apply the filter using convolution
            response = signal.convolve2d(image, gabor, mode='same', boundary='symm')
            responses.append(response)
    
    # Visualize
    fig = plt.figure(figsize=(15, 8))
    gs = fig.add_gridspec(n_wavelengths+1, n_orientations+1)
    
    # Show the original image in the top-left corner
    ax = fig.add_subplot(gs[0, 0])
    ax.imshow(image, cmap='gray')
    ax.set_title('Original Image')
    ax.axis('off')
    
    # Show the filters in the first row and column
    for i, wavelength in enumerate(wavelengths):
        for j, orientation in enumerate(orientations):
            filter_idx = i * n_orientations + j
            
            # Show filter in the first row
            if i == 0:
                ax = fig.add_subplot(gs[0, j+1])
                ax.imshow(filters[filter_idx], cmap='RdBu_r')
                ax.set_title(f'Orientation: {orientation*180/np.pi:.0f}°')
                ax.axis('off')
            
            # Show wavelength in the first column
            if j == 0:
                ax = fig.add_subplot(gs[i+1, 0])
                ax.imshow(filters[filter_idx], cmap='RdBu_r')
                ax.set_title(f'Wavelength: {wavelength}')
                ax.axis('off')
            
            # Show filter response
            ax = fig.add_subplot(gs[i+1, j+1])
            ax.imshow(np.abs(responses[filter_idx]), cmap='viridis')
            ax.axis('off')
    
    plt.tight_layout()
    plt.show()
    
    return responses

# Create a natural-like image
natural_image = create_natural_image()

# Apply Gabor filter bank
gabor_responses = apply_gabor_filter_bank(natural_image)

### Feature Extraction with Gabor Filters

As you can see from the visualization above, Gabor filters act as feature detectors that extract oriented structures from the image. Each filter responds most strongly to edges or textures that match its orientation and spatial frequency.

Some key observations:

1. **Orientation Selectivity**: Each filter responds strongly to edges aligned with its orientation and weakly to edges with perpendicular orientations

2. **Scale Selectivity**: Filters with different wavelengths (scales) detect features at different resolutions

3. **Edge Enhancement**: The filters highlight edges and boundaries in the image while suppressing uniform regions

4. **Directional Information**: The filters extract information about the directionality of features in the image

These properties make Gabor filters extremely effective at extracting meaningful features from natural scenes, which is why they serve as good models for V1 receptive fields and are also widely used in computer vision applications.

## Summary

In this notebook, we've explored the structure and properties of receptive fields in the visual cortex, with a focus on primary visual cortex (V1). We've covered:

1. **The emergence of orientation selectivity** in V1 from center-surround inputs from the LGN

2. **Gabor filters** as mathematical models of V1 receptive fields

3. **Orientation and spatial frequency tuning** properties of V1 neurons

4. **Receptive field mapping techniques** used in neuroscience research

5. **Changes in receptive field properties** across the visual hierarchy

6. **Application of Gabor filters** to natural images to extract oriented features

These receptive field properties form the foundation for more complex visual processing, including motion detection, which we'll explore in the upcoming modules. In the next section, we'll focus on simple and complex cells, which represent two distinct functional classes of neurons in V1 with different receptive field properties.

## Additional Resources

If you're interested in learning more about receptive fields in the visual cortex, here are some resources to explore:

### Papers and Books
- Hubel, D. H., & Wiesel, T. N. (1962). Receptive fields, binocular interaction and functional architecture in the cat's visual cortex. *The Journal of Physiology*, 160(1), 106-154.
  - The classic paper introducing orientation selectivity in V1

- De Valois, R. L., & De Valois, K. K. (1988). *Spatial Vision*. Oxford University Press.
  - Comprehensive book on spatial vision, including detailed treatment of receptive fields

- Jones, J. P., & Palmer, L. A. (1987). An evaluation of the two-dimensional Gabor filter model of simple receptive fields in cat striate cortex. *Journal of Neurophysiology*, 58(6), 1233-1258.
  - Key paper demonstrating the Gabor model of V1 receptive fields

### Online Resources
- [Scholarpedia: Receptive Fields](http://www.scholarpedia.org/article/Receptive_field)
  - Detailed encyclopedia article on receptive fields

- [Coursera: Neural Data Science](https://www.coursera.org/learn/neural-data-science)
  - Course covering techniques for analyzing neural data, including receptive field mapping

- [Neuronal Dynamics: From Single Neurons to Networks and Models of Cognition](https://neuronaldynamics.epfl.ch/)
  - Free online textbook with chapters on visual receptive fields